In [1]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from io import StringIO

In [2]:
#Import password
from config import username
from config import password

In [3]:
Pollution_path = "Resources/Pollution.csv"
population_path = "Resources/Population_Data.csv"
Pollution_col = ["Exposure_ID",
    "Type",
    "Country",
    "ISO3",
    "Region",
    "Name",
    "Exposure_Lower",
    "Exposure_Mean",
    "Exposure_Upper",
    "Year",
    "Pollutant",
    "Pollutant_Name",
    "Region_Name",
    "Units"
          ]

Population_col = ["Country_Name",
    "Country_Code",
    "Indicator_Name",
    "Indicator_Code",
    "year",
    "population",
          ]

In [4]:

conn = psycopg2.connect(host="localhost",dbname="pollution_analysis", user="postgres", password="Password#1", port= 5433)
cur = conn.cursor()

# Defined functions to read the csv files and write the data into PostgreSQL

In [5]:
def copy_from_stringio(conn, df, table,schema):

    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)
    cursor = conn.cursor()
    cursor.execute("SET search_path = "+schema)
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

def read_files(path,name_col,header_rows,key):
    df_set =pd.read_csv(path,names=name_col,skiprows=header_rows,quotechar='"')
    df_set.replace(',',' ', regex=True, inplace=True)
    df_set.set_index(keys=key,inplace=True)
    return df_set
    


# Read data from input files and stores in stage schema tables

In [6]:
copy_from_stringio(conn,read_files(Pollution_path,Pollution_col,1,'Exposure_ID'),'stg_pollution','stage_analysis')
copy_from_stringio(conn,read_files(population_path,Population_col,1,'Country_Name'),'stg_population','stage_analysis')

copy_from_stringio() done
copy_from_stringio() done
